In [357]:
import pandas as pd
import math
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import warnings 
from datetime import date

from scipy import stats

from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns 


from sklearn.feature_selection import f_classif # anova
from sklearn.feature_selection import chi2 # хи-квадрат
from sklearn.feature_selection import SelectPercentile

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# инструмент для создания и обучения модели
from sklearn.ensemble import RandomForestRegressor 
# инструменты для оценки точности модели
from sklearn import metrics 

In [358]:
df = pd.read_csv('./clean_data.csv')

In [359]:
df = df[df['market_cap'] < 500000000]
df = df[df['token_age_b'] <= 5]
#df = df[(df['price_change_m5'] < 1000) &(df['price_change_m5'] > 0)]

In [360]:
df.columns

Index(['token_age_b', 'transfers', 'transactions', 'buys_m5', 'sells_m5',
       'buys_h1', 'sells_h1', 'volume_m5', 'volume_h1', 'price_change_m5',
       'price_change_h1', 'price_change_h6', 'price_change_h24', 'liquidity',
       'market_cap', 'is_mutable_metadata', 'is_telegram', 'is_twitter',
       'is_website', 'boosts', 'telegram_members', 'twitter_days',
       'twitter_tweets', 'twitter_followers', 'twitter_smart_followers',
       'sns_held_all', 'sns_sold_some', 'sns_sold_all', 'PNL', 'profit',
       'sns_most_common', 'sns_most_common_except_0', 'tt_f10_bought',
       'tt_f10_sold', 'sns_f10_bought', 'sns_f10_sold', 'sns_no_bought',
       'sns_no_sold', 'tt_no_bought', 'tt_no_sold', 'sns_pnl_loss',
       'tt_pnl_loss'],
      dtype='object')

In [361]:
x = df.drop(columns=[
    # 'Unnamed: 53',
    #    'Unnamed: 54',
    
    "profit", "transfers", "transactions",
    'price_change_h6', 'price_change_h24',
    'volume_h1',
    'price_change_h1',
    'buys_h1', 'sells_h1',
    'PNL',
    # 'telegram_members', 'twitter_days', 
    # 'twitter_tweets', 'twitter_followers', 'twitter_smart_followers', 
    # 'sns_most_common', 'sns_most_common_except_0',
    #    'tt_f10_bought', 'tt_f10_sold', 'sns_f10_bought', 'sns_no_bought',
    #    'sns_no_sold', 'tt_no_bought', 'tt_no_sold', 'sns_pnl_loss',
    #    'tt_pnl_loss'
    ])
y = df["profit"].copy()

In [362]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 603 entries, 0 to 2728
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   token_age_b               603 non-null    float64
 1   buys_m5                   603 non-null    float64
 2   sells_m5                  603 non-null    float64
 3   volume_m5                 603 non-null    float64
 4   price_change_m5           603 non-null    float64
 5   liquidity                 603 non-null    float64
 6   market_cap                603 non-null    float64
 7   is_mutable_metadata       603 non-null    int64  
 8   is_telegram               603 non-null    int64  
 9   is_twitter                603 non-null    int64  
 10  is_website                603 non-null    int64  
 11  boosts                    603 non-null    int64  
 12  telegram_members          603 non-null    int64  
 13  twitter_days              603 non-null    int64  
 14  twitter_tweets

In [363]:
# Объявляем вспомогательную функцию по определению наличия выбросов в признаке
def there_are_outliers(data, feature) -> bool:
    '''
    Функция определяющая методом Тьюки наличие выбросов в признаке
    Args:
        data (DataFrame): датасет
        feature (str): наименование признака
    Returns:
        boolean: возвращает True если выбросы в признаке есть в наличии 
    '''
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return x.min() < lower_bound or x.max() > upper_bound

In [364]:
warnings.simplefilter("ignore", UserWarning)

In [365]:
alpha = 0.05 

scale_columns = [
    'token_age_b', 
    #'transfers', 'transactions', 
    'buys_m5', 'sells_m5',
       #'buys_h1', 'sells_h1', 
       'volume_m5', 
       #'volume_h1', 
       'price_change_m5',
       #'price_change_h1', 
    #    'price_change_h6', 'price_change_h24', 
       'liquidity',
       'market_cap', 
       'boosts', 
       'telegram_members', 'twitter_days',
       'twitter_tweets', 'twitter_followers', 'twitter_smart_followers',
       'sns_held_all', 'sns_sold_some', 'sns_sold_all',
       'tt_f10_bought', 'tt_f10_sold', 'sns_f10_bought', 'sns_f10_sold', 'sns_no_bought',
       'sns_no_sold', 'tt_no_bought', 'tt_no_sold', 'sns_pnl_loss',
       'tt_pnl_loss', 'sns_most_common'
]
s = []
# Приступаем к маштабированию признаков,
# исходные признаки не трогаем, создаем отдельные,
# т.к. код в данной ячейке может быть исполнен повторно
for feature in scale_columns:
    # С помощью теста Шапиро-Уилка проверим, нормально ли распределён признак 
    result = stats.shapiro(x[feature])
    if result[1] <= alpha:
        print('Признак {} распределен не нормально,'.format(feature))
        # С помощью метода Тьюки проверим, есть ли выбросы
        if there_are_outliers(x, feature):
            print('в признаке {} имеются выбросы, используем RobustScaler().'
                  .format(feature))
            r_scaler = preprocessing.RobustScaler()
            x[str(feature) + '_scaled'] = r_scaler.fit_transform(
                x[feature].values.reshape(x.shape[0],1))
        else:
            print('в признаке {} нет выбросов, используем MinMaxScaler().'
                  .format(feature))
            mm_scaler = preprocessing.MinMaxScaler()
            x[str(feature) + '_scaled'] = mm_scaler.fit_transform(
                x[feature].values.reshape(x.shape[0],1))
    else:
        print('Признак {} распределен нормально,'.format(feature))
        print('используем StandardScaler().')
        s_scaler = preprocessing.StandardScaler()
        x[str(feature) + '_scaled'] = s_scaler.fit_transform(
            x[feature].values.reshape(x.shape[0],1))  
    s.append(str(feature) + '_scaled')



Признак token_age_b распределен не нормально,
в признаке token_age_b нет выбросов, используем MinMaxScaler().
Признак buys_m5 распределен не нормально,
в признаке buys_m5 имеются выбросы, используем RobustScaler().
Признак sells_m5 распределен не нормально,
в признаке sells_m5 имеются выбросы, используем RobustScaler().
Признак volume_m5 распределен не нормально,
в признаке volume_m5 имеются выбросы, используем RobustScaler().
Признак price_change_m5 распределен не нормально,
в признаке price_change_m5 имеются выбросы, используем RobustScaler().
Признак liquidity распределен не нормально,
в признаке liquidity имеются выбросы, используем RobustScaler().
Признак market_cap распределен не нормально,
в признаке market_cap имеются выбросы, используем RobustScaler().
Признак boosts распределен не нормально,
в признаке boosts имеются выбросы, используем RobustScaler().
Признак telegram_members распределен не нормально,
в признаке telegram_members имеются выбросы, используем RobustScaler().
Пр

In [366]:
x = x.drop(scale_columns, axis=1)

In [367]:
x.head()

,is_mutable_metadata,is_telegram,is_twitter,is_website,sns_most_common_except_0,token_age_b_scaled,buys_m5_scaled,sells_m5_scaled,volume_m5_scaled,price_change_m5_scaled,...,tt_f10_sold_scaled,sns_f10_bought_scaled,sns_f10_sold_scaled,sns_no_bought_scaled,sns_no_sold_scaled,tt_no_bought_scaled,tt_no_sold_scaled,sns_pnl_loss_scaled,tt_pnl_loss_scaled,sns_most_common_scaled
0,1,0,0,0,0.1,0.312693,-0.230415,-0.253602,0.528993,656.520764,...,1.083561,2.737355,2.524092,0.254545,0.850575,0.151515,0.806818,-0.133333,1.714286,0.1
2,1,0,0,0,0.1,0.541796,0.161290,-0.057637,0.888933,427.138819,...,1.688067,2.733371,2.432012,0.254545,0.873563,0.171717,0.761364,-0.333333,1.285714,0.1
3,0,0,0,0,11.0,0.486068,0.474654,0.772334,1.048213,1.638819,...,0.908028,0.222631,0.296238,0.218182,0.298851,0.929293,0.000000,0.200000,-0.142857,0.0
4,0,0,0,0,23.0,0.430341,-0.096774,0.028818,-0.274718,0.279653,...,-0.161959,-0.048677,-0.313166,0.163636,0.402299,0.111111,0.306818,1.666667,1.857143,0.0
5,0,0,0,0,23.0,0.696594,0.815668,0.824207,0.170882,0.302222,...,0.193476,-0.283082,-0.525135,0.109091,0.528736,0.070707,0.011364,-0.066667,-0.142857,23.0


In [368]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [369]:
x_train = torch.tensor(x_train.values).to(torch.float32)
x_test = torch.tensor(x_test.values).to(torch.float32)
y_train = torch.tensor(y_train.values).to(torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test.values).to(torch.float32).unsqueeze(1)

In [370]:
class LogisticRegression(torch.nn.Module):

    def __init__(self, n_features):
        super(LogisticRegression, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out

In [371]:
n_features = x_train.shape[1]
model = LogisticRegression(n_features)

optim = torch.optim.SGD(model.parameters(), lr=1)
criterion = torch.nn.BCELoss()

In [372]:
EPOCHS = 5

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        print(f'Среднее значение функции потерь за эпоху {e}: {loss.data}')
    return model

In [373]:
model = train(model, optim, criterion, x_train, y_train)

Среднее значение функции потерь за эпоху 1: 1.7133110761642456
Среднее значение функции потерь за эпоху 2: 68.32557678222656
Среднее значение функции потерь за эпоху 3: 68.32534790039062
Среднее значение функции потерь за эпоху 4: 68.32511138916016
Среднее значение функции потерь за эпоху 5: 68.32488250732422


In [374]:
def accuracy(model, x, y):
    out = model(x)
    correct = torch.abs(y - out) < 0.5
    return correct.float().mean()

plain_accuracy = accuracy(model, x_test, y_test)
print(f'Точность модели: {plain_accuracy}')

Точность модели: 0.3636363744735718


In [375]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

x = df.drop(columns=["profit"])
y = df["profit"].copy()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Создание и обучение модели логистической регрессии
model = LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(X_train, y_train)

# Предсказание классов на тестовой выборке
y_pred = model.predict(X_test)

# Вычисление точности классификации
accuracy = accuracy_score(y_test, y_pred)
print("Точность классификации: {:.2f}%".format(accuracy * 100))

Точность классификации: 95.04%


In [376]:
from sklearn.metrics import classification_report  
from sklearn.metrics import accuracy_score  
from sklearn.neighbors import KNeighborsClassifier  

In [377]:
KNN_model = KNeighborsClassifier(n_neighbors=10)

In [378]:
KNN_model.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [379]:
KNN_prediction = KNN_model.predict(x_test)

In [380]:
print(accuracy_score(KNN_prediction, y_test))  
print(classification_report(KNN_prediction, y_test))

0.6694214876033058
              precision    recall  f1-score   support

           0       0.96      0.67      0.79       112
           1       0.14      0.67      0.23         9

    accuracy                           0.67       121
   macro avg       0.55      0.67      0.51       121
weighted avg       0.90      0.67      0.75       121

